# 🧠 SVM From Scratch with Gradient Descent

This guide walks through the implementation of a **linear Support Vector Machine (SVM)** from scratch in Python using only `numpy`. The model uses **hinge loss** with **L2 regularization** and optimizes the objective using **stochastic gradient descent (SGD)**.

---

## 📌 Objective Function

The primal form of the SVM optimization problem is:

$$
\min_{\mathbf{w}, b} \ \lambda \cdot \frac{1}{2} \|\mathbf{w}\|^2 + \frac{1}{n} \sum_{i=1}^n \max\left(0, 1 - y_i(\mathbf{w}^\top \mathbf{x}_i + b)\right)
$$

- $\mathbf{w}$ — weight vector  
- $b$ — bias  
- $\lambda$ — regularization parameter  
- $y_i \in \{-1, +1\}$ — class labels  
- $\mathbf{x}_i \in \mathbb{R}^d$ — feature vector

---

## 📐 Gradient Derivation

### Regularization Term

$$
\frac{\partial}{\partial \mathbf{w}} \left( \lambda \cdot \frac{1}{2} \|\mathbf{w}\|^2 \right) = \lambda \mathbf{w}
$$

$$
\frac{\partial}{\partial b} \left( \lambda \cdot \frac{1}{2} \|\mathbf{w}\|^2 \right) = 0
$$

---

### Hinge Loss Gradient

If constraint is satisfied: $y_i(\mathbf{w}^\top \mathbf{x}_i + b) \geq 1$

$$
\frac{\partial L}{\partial \mathbf{w}} = \lambda \mathbf{w}, \quad
\frac{\partial L}{\partial b} = 0
$$

If constraint is violated: $y_i(\mathbf{w}^\top \mathbf{x}_i + b) < 1$

$$
\frac{\partial L}{\partial \mathbf{w}} = \lambda \mathbf{w} - y_i \mathbf{x}_i, \quad
\frac{\partial L}{\partial b} = -y_i
$$

---

## ✅ Summary

- Uses binary labels $y \in \{-1, 1\}$
- Hinge loss only penalizes if prediction is wrong or within margin
- Regularization keeps weights small and prevents overfitting


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import numpy as np

In [2]:
X, y = make_blobs(
    n_samples=250, n_features=2, centers=2, cluster_std=1.05, random_state=1
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=1)


In [3]:
class SVM:
  def __init__(self,learning_rate=1e-3, lambda_param=1e-2, n_iters=1000):
    self.learning_rate=learning_rate
    self.lambda_param=lambda_param
    self.n_iters=n_iters
    self.w=None
    self.b=None

  def _init_weights_bias(self, X):
    n_features=X.shape[1]
    self.w=np.zeros(n_features)
    self.b=0

  def _get_cls_map(self, y):
    return np.where(y<=0,-1,1)

  def _satisfy_constraint(self,x,idx):
      linear_model=np.dot(x,self.w)+self.b
      return self.cls_map[idx] * linear_model >= 1

  def _get_gradients(self,constraint,x,idx):
    if constraint:
      dw=self.lambda_param*self.w
      db=0
      return dw,db

    dw=self.lambda_param*self.w-np.dot(self.cls_map[idx],x)
    db=-self.cls_map[idx]
    return dw,db

  def _update_weights_bias(self,dw,db):
    self.w-=self.learning_rate*dw
    self.b-=self.learning_rate*db


  def fit(self, X, y):
        self._init_weights_bias(X)
        self.cls_map = self._get_cls_map(y)

        for _ in range(self.n_iters):
            for idx, x in enumerate(X):
                constrain = self._satisfy_constraint(x, idx)
                dw, db = self._get_gradients(constrain, x, idx)
                self._update_weights_bias(dw, db)

  def predict(self, X):
      estimate = np.dot(X, self.w) + self.b
      prediction = np.sign(estimate)
      return np.where(prediction == -1, 0, 1)


In [4]:
X, y = make_blobs(
    n_samples=250, n_features=2, centers=2, cluster_std=1.05, random_state=1
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=1)

clf = SVM(n_iters=1000)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true==y_pred) / len(y_true)
    return accuracy

print("SVM Accuracy: ", accuracy(y_test, predictions))

SVM Accuracy:  1.0
